In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
def extract(frame):
    # Load the MediaPipe Hands model
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    lm = np.zeros((2, 21, 3))
    for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
        for j, landmark in enumerate(hand_landmarks.landmark):
            lm[i][j] = [landmark.x, landmark.y, landmark.z]    
    hands.close()

    return lm

In [3]:
model = load_model('Models/gesture_model.keras')

In [4]:
mapping = {
    0: 'closedfist',
    1: 'four',
    2: 'openpalm',
    3: 'pointup',
    4: 'three',
    5: 'thumbsdown',
    6: 'thumbsup',
    7: 'victory'
}

In [6]:
import cv2
import numpy as np
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="Models/converted_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_frame(frame):
    processed_frame = extract(frame).reshape(-1,42,3)
    return processed_frame

cap = cv2.VideoCapture(0)
text = 'No Gesture Yet'
if not cap.isOpened():
    print("Error: Couldn't open webcam")
    exit()

frame_count = 0

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Couldn't read frame")
        break

    frame_count += 1

    if frame_count % 30 != 0: 
        cv2.putText(frame,
            text,
            org=(50, 50),  
            fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
            fontScale=1, 
            color=(0, 0, 255), 
            thickness=2)

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    try:
        processed_frame = preprocess_frame(frame)
        
        interpreter.set_tensor(input_details[0]['index'], np.array(processed_frame, dtype=np.float32))

        interpreter.invoke()

        output_data = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = np.argmax(output_data)

        text = mapping[predicted_class]

    except Exception as e:
        print("Error:", e)
        continue

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable
Error: 'NoneType' object is not iterable


In [11]:
text

'victory'

In [15]:
type(mapping[test])

str